<a href="https://colab.research.google.com/github/jaeryong77/NLP_sentimental/blob/main/%5B%EB%85%BC%EB%AC%B8%EC%9D%B4%EC%9A%A9%5D_%EB%84%A4%EC%9D%B4%EB%B2%84%EB%89%B4%EC%8A%A4%EC%A0%9C%EB%AA%A9_%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%86%8C%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''

메인로직 참조 소스 , 언론사별 테그
: https://everyday-tech.tistory.com/entry/3%ED%83%84-%EC%89%BD%EA%B2%8C-%EB%94%B0%EB%9D%BC%ED%95%98%EB%8A%94-%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%89%B4%EC%8A%A4-%ED%81%AC%EB%A1%A4%EB%A7%81-%EB%B3%B8%EB%AC%B8-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0

html find , bs.find 소스
https://arehoow.tistory.com/9 

내용정제를 위한 참고 소스
https://bumcrush.tistory.com/116



In [ ]:


#언론사 코드
press_code ={'donga':'1020', 'kookmin':'1005', 'kyenhyang':'1032','joonang':'1025', 'hangeyre':'1028','chosun':'1023', 'moonhwa':'1021'}
press_code.keys()
for press in press_code.keys():
  print(press_code.get(press))
print(press_code.values())

1020
1005
1032
1025
1028
1023
1021
dict_values(['1020', '1005', '1032', '1025', '1028', '1023', '1021'])


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re

In [ ]:
# 크롤링 입력조건 입력
###################################
# 여기서 부터 진짜 실행 코드 
##################################
print('크롤링 실행 입력조건\n')
keyword = str(input('keyword:'))
sdate = str(input('s.date:'))
edate = str(input('e.date:'))
sdate1 = sdate[0:4]+'.'+sdate[4:6]+'.'+sdate[6:8]
edate1 = edate[0:4]+'.'+edate[4:6]+'.'+edate[6:8]
print('\n\n입력조건확인\nkeyword : {} \n sdate : {} \n edate : {} \n sdate1 : {} \n edate1 : {} '.format(keyword,sdate,edate,sdate1,edate1))
start = '1'

#언론사 구분 없음
# base_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=0&photo=0&field=0&pd=3&ds={}&de={}&cluster_rank=20&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{}to{},a:all&start={}'
#언론사별
base_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=0&photo=0&field=0&pd=3&ds={}&de={}&cluster_rank=0&mynews=1&office_type=1&office_section_code=2&news_office_checked={}&nso=so:r,p:from{}to{},a:all&start={}'

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}

url = base_url.format(keyword,sdate,edate,'1020',sdate1,edate1,start)
print(url)

# 언론사 코드 = kbs 1056 , sbs 1055 , mbc = 1214


크롤링 실행 입력조건

keyword:코로나19
s.date:20210801
e.date:20210831


입력조건확인
keyword : 코로나19 
 sdate : 20210801 
 edate : 20210831 
 sdate1 : 2021.08.01 
 edate1 : 2021.08.31 
https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나19&sort=0&photo=0&field=0&pd=3&ds=20210801&de=20210831&cluster_rank=0&mynews=1&office_type=1&office_section_code=2&news_office_checked=1020&nso=so:r,p:from2021.08.01to2021.08.31,a:all&start=1


In [ ]:
title_list = []
url_list=[]
tmp_url = {}

for p_code in press_code.values() :
  start = 1
  title_list=[] 
  url_list=[]
  date_list=[]
  for i in range(1,5): # 10 -> 100개, 30 -> 300개
    try :
      url = base_url.format(keyword,sdate,edate,p_code,sdate1,edate1,start)
      print(url)      
      raw = requests.get(url, headers=headers)
      soup = bs(raw.text, 'html.parser')  
      news_title = [title['title'] for title in soup.find_all('a', attrs={'class':'news_tit'})]
      news_url = [ url['href'] for url in soup.find_all('a', attrs={'class':'news_tit'}) ] # 기사 url
      dates = [date.get_text() for date in soup.find_all('span', attrs={'class':'info'})]
      news_dates = []
      for date in dates:
        if re.search(r'\d+.\d+.\d+.', date) != None :
          news_dates.append(date)
      title_list = title_list+news_title
      url_list = url_list + news_url
      date_list = date_list+news_dates
      start += 10
    except :
      print("stop됨",start)
      break
# 언론사별 data frame 을 만든다 
  # df_columns = ['title','url']
  tmp_dic = {}
  tmp_dic = {'title' : title_list, 'url': url_list, 'date': date_list}
  globals()['df_{}'.format(p_code)] = pd.DataFrame(tmp_dic)
  

https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나19&sort=0&photo=0&field=0&pd=3&ds=20210801&de=20210831&cluster_rank=0&mynews=1&office_type=1&office_section_code=2&news_office_checked=1020&nso=so:r,p:from2021.08.01to2021.08.31,a:all&start=1
https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나19&sort=0&photo=0&field=0&pd=3&ds=20210801&de=20210831&cluster_rank=0&mynews=1&office_type=1&office_section_code=2&news_office_checked=1020&nso=so:r,p:from2021.08.01to2021.08.31,a:all&start=11
https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나19&sort=0&photo=0&field=0&pd=3&ds=20210801&de=20210831&cluster_rank=0&mynews=1&office_type=1&office_section_code=2&news_office_checked=1020&nso=so:r,p:from2021.08.01to2021.08.31,a:all&start=21
https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나19&sort=0&photo=0&field=0&pd=3&ds=20210801&de=20210831&cluster_rank=0&mynews=1&office_type=1&office_section_code=2&news_office_checked=1020&nso=so

In [ ]:
df_1021.count()

title    40
url      40
date     40
dtype: int64

In [ ]:
# #언론사 코드
# press_code ={'donga':'1020', 'kookmin':'1005', 'kyenhyang':'1032','joonang':'1025', 'hangeyre':'1028','chosun':'1023', 'moonhwa':'1021'}
# press_code.keys()
# 엑셀 만들기

filenm = 'df_1020_'+sdate[0:6]+'.xlsx'
df_1020.to_excel(filenm)

filenm = 'df_1005_'+sdate[0:6]+'.xlsx'
df_1005.to_excel(filenm)

filenm = 'df_1032_'+sdate[0:6]+'.xlsx'
df_1032.to_excel(filenm)

filenm = 'df_1025_'+sdate[0:6]+'.xlsx'
df_1025.to_excel(filenm)

filenm = 'df_1028_'+sdate[0:6]+'.xlsx'
df_1028.to_excel(filenm)

filenm = 'df_1023_'+sdate[0:6]+'.xlsx'
df_1023.to_excel(filenm)

filenm = 'df_1021_'+sdate[0:6]+'.xlsx'
df_1021.to_excel(filenm)

In [ ]:
df_1020.count()

title    466
url      466
dtype: int64

In [ ]:
################################################################################################333
# 이후는 이전 프로그램임 
##################################################################################################

In [ ]:
# 크롤링 펑션
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = bs(req.text, 'html.parser')
    result = [] # 0 = text , 1 = 날자

    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('div', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
        
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        # text = soup.find('div', {'itemprop' : 'articleBody'}).text
        result.append(soup.find('div', {'itemprop' : 'articleBody'}).text)

    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        # text = soup.find('div', {'itemprop' : 'articleBody'}).text
        result.append(soup.find('div', {'itemprop' : 'articleBody'}).text)
        date = soup.find('span',{'class':'date'})
        result.append(date.find('span').text)
    # KBS
    elif 'news.kbs' in url:
        # text = soup.find('div', {'id' : 'cont_newstext'}).text
        result.append(soup.find('div', {'id' : 'cont_newstext'}).text)
        date = date = soup.find('span',{'class':'txt-info'}  ) #날자 
        result.append(date.find('em', {'class': 'date'}).text)

    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 그 외
    else:
        #  text = None
        result = []   
    # if text != None :    
    if len(result) != 0:
      # return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
      if result[0] is not None :
        result[0].replace('\n','').replace('\r','').replace('<br>','').replace('\t','')      
      return result

In [ ]:
# ulr_list 에 각언론사별 100건씩 url 리스트가  들어온다. 

#최종 저장소

idx = 1
news_dict = {}
crawling_result =[] #+
for url_item in url_list:
  crawling_result = crawling_main_text(url_item)   #+
  # news_dict[idx] = { 'url' : url_item,
  #                   'text' : crawling_main_text(url_item) }
  news_dict[idx] = { 'url' : url_item,
                    'text' : crawling_result[0],
                    'date' : crawling_result[1]}


  idx += 1

news_df = pd.DataFrame(news_dict).T
news_df.to_excel("naver_news.xlsx")




In [ ]:
#300 개 저장하는데 5분 30초 걸림


In [ ]:
    req = requests.get('https://news.kbs.co.kr/news/view.do?ncd=5152826&ref=A')
    req.encoding = None
    soup = bs(req.text, 'html.parser')

    text = soup.find('div', {'id' : 'cont_newstext'}).text
    date = soup.find('span',{'class':'txt-info'})
    input_date = date.find('em', {'class': 'date'}).text


    # import re
    # input_date = re.search(r'\d+.\d+.\d+', date)
    


In [ ]:
input_date

'입력 2021.04.01 (12:46)'

In [ ]:
    req = requests.get('https://news.sbs.co.kr/news/endPage.do?news_id=N1006264727&plink=ORI&cooper=NAVER')
    req.encoding = None
    soup = bs(req.text, 'html.parser')

    date = soup.find('span',{'class':'date'})
    date1 = date.find('span')
    print(date1)

<span>2021.04.01 17:33</span>


In [ ]:
x = None
print(type(x))
if x is not None:
  print('T')

<class 'NoneType'>


In [ ]:
test = {'title' : [], 'text' : []}


In [ ]:
test['title'] = ['1','2']
test

{'text': [], 'title': ['1', '2']}

In [ ]:
print(type(test['title']))

<class 'list'>
